<a href="https://colab.research.google.com/github/sorawit-kamlangsub/KexExportIYPNB/blob/main/KexExport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install flask httpx openpyxl pyngrok

In [ ]:
%%writefile app.py
import asyncio
import threading
import uuid
from dataclasses import dataclass, field
from typing import List, Dict, Optional

from flask import Flask, jsonify, request, render_template_string
import httpx

app = Flask(__name__)

# -----------------------------
# HTML (UI แบบประหยัดแบนด์วิธ: โชว์แค่ progress, กด Export ค่อยดึง rows ทีเดียว)
# -----------------------------
INDEX_HTML = r"""
<!doctype html>
<html lang="th">
<head>
  <meta charset="utf-8" />
  <title>Kex Shipment Tracking Export (Flask + ngrok)</title>
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <script src="https://cdn.jsdelivr.net/npm/xlsx/dist/xlsx.full.min.js"></script>
  <style>
    body { font-family: sans-serif; padding: 20px; max-width: 720px; margin: 0 auto; }
    h1 { margin-bottom: 8px; }
    label { display:block; margin-top: 12px; font-weight: 600; }
    input[type="text"], input[type="password"] { width: 100%; padding: 10px 12px; border:1px solid #ccc; border-radius: 6px; }
    .row { display:flex; gap: 10px; margin-top: 12px; flex-wrap: wrap; }
    button {
      padding: 10px 16px; border: none; border-radius: 8px; color: #fff; cursor: pointer; font-size: 15px;
    }
    #downloadBtn { background: #1565c0; }
    #stopBtn { background: #ef6c00; }
    #exportBtn { background: #2e7d32; }
    button:disabled { background: #9e9e9e; cursor: not-allowed; }
    #status { margin-top: 10px; font-weight: 600; }
    #error { color: #c62828; margin-top: 8px; white-space: pre-wrap; }
  </style>
</head>
<body>
  <h1>Kex React TS → Flask Export</h1>
  <p>ฝั่งเซิร์ฟเวอร์โหลดข้อมูล พร้อมหยุดได้เมื่อ error; หน้าบราว์เซอร์ดึงแค่ progress และ export ตอนจบ</p>

  <label>TOKEN</label>
  <input id="token" type="password" placeholder="ใส่ Bearer Token" />

  <label>StartDate (YYYYMMDD)</label>
  <input id="startDate" type="text" value="" />

  <label>EndDate (YYYYMMDD)</label>
  <input id="endDate" type="text" value="" />

  <div class="row">
    <button id="downloadBtn">⬇️ ดาวน์โหลดข้อมูล</button>
    <button id="stopBtn" disabled>⛔ หยุดการโหลด</button>
    <button id="exportBtn" disabled>⬇️ ส่งออก Excel</button>
  </div>

  <p id="status">โปรดกดดาวน์โหลดข้อมูล</p>
  <p id="error"></p>

<script>
const $ = (id) => document.getElementById(id);
const statusEl = $("status");
const errorEl = $("error");
const exportBtn = $("exportBtn");
const downloadBtn = $("downloadBtn");
const stopBtn = $("stopBtn");

let jobId = null;
let pollTimer = null;

// default วันนี้
(function setDefaultDates(){
  const now = new Date();
  const yyyy = now.getFullYear();
  const mm = String(now.getMonth()+1).padStart(2,'0');
  const dd = String(now.getDate()).padStart(2,'0');
  $("endDate").value = `${yyyy}${mm}${dd}`;
  $("startDate").value = `${yyyy}${mm}${dd}`;
})();

function setLoadingUIActive(active) {
  downloadBtn.disabled = active;
  stopBtn.disabled = !active;
  exportBtn.disabled = active; // จะเปิดหลังจบ/ยกเลิก
}

function renderStatus(done, total, stateText) {
  if (total > 0) statusEl.textContent = `${stateText} (${done} / ${total})`;
  else statusEl.textContent = stateText;
}

async function startJob() {
  errorEl.textContent = "";
  const token = $("token").value.trim();
  const startDate = $("startDate").value.trim();
  const endDate = $("endDate").value.trim();
  if (!token || !startDate || !endDate) {
    errorEl.textContent = "กรุณากรอก TOKEN / StartDate / EndDate ให้ครบ";
    return;
  }

  setLoadingUIActive(true);
  renderStatus(0, 0, "กำลังเริ่มงาน...");

  try {
    const res = await fetch("/api/start", {
      method: "POST",
      headers: {"Content-Type":"application/json"},
      body: JSON.stringify({ token, startDate, endDate })
    });
    if (!res.ok) throw new Error("Start HTTP " + res.status);
    const data = await res.json();
    jobId = data.job_id;

    pollTimer = setInterval(pollProgress, 1000);
  } catch (e) {
    errorEl.textContent = "เริ่มงานไม่สำเร็จ: " + e;
    setLoadingUIActive(false);
  }
}

async function pollProgress() {
  if (!jobId) return;
  try {
    const res = await fetch(`/api/progress/${jobId}`);
    if (!res.ok) throw new Error("Progress HTTP " + res.status);
    const p = await res.json();

    renderStatus(p.done, p.total, p.state);
    if (p.error) errorEl.textContent = p.error;

    if (p.finished || p.stopped || p.error) {
      clearInterval(pollTimer);
      pollTimer = null;
      setLoadingUIActive(false);
      // ถ้าต้องการบล็อก export เมื่อ error ให้ใช้: exportBtn.disabled = !!p.error;
      exportBtn.disabled = false;
      if (p.finished) renderStatus(p.done, p.total, "โหลดข้อมูลเสร็จสิ้น");
      if (p.stopped) renderStatus(p.done, p.total, "ยกเลิกแล้ว");
    }
  } catch (e) {
    errorEl.textContent = "ดึงความคืบหน้าไม่สำเร็จ: " + e;
  }
}

async function stopJob() {
  if (!jobId) return;
  try { await fetch(`/api/stop/${jobId}`, { method: "POST" }); } catch {}
}

function exportToExcel(rows) {
  if (!rows.length) {
    alert("ไม่มีข้อมูลให้ส่งออก");
    return;
  }

  // ลำดับคอลัมน์ให้ตรงกับ HTML ต้นฉบับ
  const headers = [
    "consignmentNo",
    "loc",
    "s_date",
    "s_time",
    "s_desc",
    "d_date",
    "d_time",
    "d_desc"
  ];

  // แปลง rows -> array-of-arrays ตามลำดับ headers
  const data = rows.map(r => headers.map(h => r?.[h] ?? ""));

  // สร้างชีตด้วย AOA เพื่อบังคับลำดับคอลัมน์
  const ws = XLSX.utils.aoa_to_sheet([headers, ...data]);
  const wb = XLSX.utils.book_new();
  XLSX.utils.book_append_sheet(wb, ws, "Tracking");

  const now = new Date();
  const yyyy = now.getFullYear();
  const mm = String(now.getMonth()+1).padStart(2,'0');
  const dd = String(now.getDate()).padStart(2,'0');
  const hh = String(now.getHours()).padStart(2,'0');
  const mi = String(now.getMinutes()).padStart(2,'0');
  const filename = `tracking_kex_${yyyy}-${mm}-${dd}_${hh}-${mi}.xlsx`;

  XLSX.writeFile(wb, filename);
}

async function doExport() {
  if (!jobId) {
    alert("ยังไม่มีงานที่เสร็จสิ้น");
    return;
  }
  try {
    const res = await fetch(`/api/rows/${jobId}`);
    if (!res.ok) throw new Error("Rows HTTP " + res.status);
    const js = await res.json();
    exportToExcel(js.rows || []);
  } catch (e) {
    errorEl.textContent = "ดึงข้อมูลสำหรับส่งออกไม่สำเร็จ: " + e;
  }
}

downloadBtn.addEventListener("click", startJob);
stopBtn.addEventListener("click", stopJob);
exportBtn.addEventListener("click", doExport);
</script>
</body>
</html>
"""

# -----------------------------
# งานดึงข้อมูล (async + background thread) พร้อม "หยุดเมื่อ error"
# -----------------------------
LIST_URL_TPL = "https://cms.th.kex-express.com/easyship-api/api/Report/v2.5/Report/Shipment/{start}/{end}"
TRACK_URL_TPL = "https://cms.th.kex-express.com/easyship-api/api/Report/v2.6/Tracking/{consignment}/th"

CONCURRENCY = 5
TIMEOUT_SECS = 30.0

async def fetch_shipments(client: httpx.AsyncClient, token: str, start: str, end: str) -> List[str]:
    url = LIST_URL_TPL.format(start=start, end=end)
    headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
    r = await client.get(url, headers=headers, timeout=TIMEOUT_SECS)
    r.raise_for_status()
    data = r.json()
    shipments = []
    if isinstance(data, dict):
        if isinstance(data.get("Shipment"), list): shipments = data["Shipment"]
        elif isinstance(data.get("result"), list): shipments = data["result"]
    cons = [s.get("ConsignmentNo") for s in shipments if isinstance(s, dict) and s.get("ConsignmentNo")]
    return cons

def extract_events(json_obj: dict) -> List[dict]:
    if not isinstance(json_obj, dict):
        return []
    if "result" in json_obj and isinstance(json_obj["result"], dict) and "shipment_status" in json_obj["result"]:
        return json_obj["result"]["shipment_status"] or []
    if "data" in json_obj and isinstance(json_obj["data"], list):
        return json_obj["data"] or []
    return []

def pick_delivered_event(events: List[dict]) -> Optional[dict]:
    for e in events:
        if e.get("s_desc") == "จัดส่งพัสดุสำเร็จ":
            return e
    return None

def select_create_branch_events(events: List[dict]) -> List[dict]:
    return [e for e in events if e.get("s_desc") == "ผู้ส่งมาส่งพัสดุที่สาขา"]

# โยน error ขึ้นไปให้ตัวควบคุมหยุดงานทั้งก้อน
async def fetch_tracking_for_cons(client: httpx.AsyncClient, token: str, consignment_no: str) -> List[dict]:
    url = TRACK_URL_TPL.format(consignment=consignment_no)
    headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
    r = await client.get(url, headers=headers, timeout=TIMEOUT_SECS)
    r.raise_for_status()

    events = extract_events(r.json())
    create_events = select_create_branch_events(events)
    delivered = pick_delivered_event(events)

    d_date = delivered.get("s_date", "") if delivered else ""
    d_time = delivered.get("s_time", "") if delivered else ""
    d_desc = delivered.get("s_desc", "") if delivered else ""

    if not create_events:
        return [{
            "consignmentNo": consignment_no,
            "loc": "-",
            "s_date": "-",
            "s_time": "-",
            "s_desc": "ไม่มีข้อมูล ผู้ส่งมาส่งพัสดุที่สาขา",
            "d_date": d_date, "d_time": d_time, "d_desc": d_desc
        }]

    rows = []
    for e in create_events:
        rows.append({
            "consignmentNo": consignment_no,
            "loc": e.get("loc", ""),
            "s_date": e.get("s_date", ""),
            "s_time": e.get("s_time", ""),
            "s_desc": e.get("s_desc", ""),
            "d_date": d_date, "d_time": d_time, "d_desc": d_desc
        })
    return rows

@dataclass
class Job:
    token: str
    start: str
    end: str
    total: int = 0
    done: int = 0
    rows: List[dict] = field(default_factory=list)
    stop_flag: bool = False
    finished: bool = False
    error: Optional[str] = None
    state: str = "กำลังเตรียมรายการ..."

JOBS: Dict[str, Job] = {}
JOBS_LOCK = threading.Lock()

def run_job_bg(job_id: str):
    job = JOBS.get(job_id)
    if not job:
        return

    async def runner():
        try:
            limits = httpx.Limits(max_keepalive_connections=10, max_connections=10)
            async with httpx.AsyncClient(limits=limits) as client:
                # 1) รายการ shipment
                with JOBS_LOCK:
                    job.state = "กำลังดึงรายการ shipment..."
                cons_list = await fetch_shipments(client, job.token, job.start, job.end)

                with JOBS_LOCK:
                    job.total = len(cons_list)
                    job.state = "กำลังโหลดข้อมูล..."

                sem = asyncio.Semaphore(CONCURRENCY)

                async def one_cons(cons_no: str):
                    if job.stop_flag:
                        return
                    async with sem:
                        if job.stop_flag:
                            return
                        try:
                            rows = await fetch_tracking_for_cons(client, job.token, cons_no)
                        except Exception as ex:
                            # เจอ error: หยุดงานทั้งก้อน
                            with JOBS_LOCK:
                                job.error = f"หยุดโหลดเพราะเกิดข้อผิดพลาดที่ {cons_no}: {ex}"
                                job.stop_flag = True
                                job.state = "เกิดข้อผิดพลาด (กำลังยกเลิก...)"
                            return
                        with JOBS_LOCK:
                            job.rows.extend(rows)
                            job.done += 1

                tasks = [one_cons(c) for c in cons_list]

                # ไล่ทีละก้อน เพื่อเช็ค stop_flag ได้บ่อย
                CHUNK = 100
                for i in range(0, len(tasks), CHUNK):
                    if job.stop_flag:
                        break
                    chunk = tasks[i:i+CHUNK]
                    await asyncio.gather(*chunk)
                    if job.stop_flag:
                        break

            with JOBS_LOCK:
                if job.stop_flag and job.error:
                    job.finished = False
                    job.state = "ยกเลิกแล้ว (เกิดข้อผิดพลาด)"
                else:
                    job.finished = True
                    job.state = "โหลดข้อมูลเสร็จสิ้น"

        except Exception as ex:
            with JOBS_LOCK:
                job.error = f"เกิดข้อผิดพลาดระดับระบบ: {ex}"
                job.stop_flag = True
                job.finished = False
                job.state = "ยกเลิกแล้ว (เกิดข้อผิดพลาด)"

    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(runner())
    loop.close()

# -----------------------------
# Routes
# -----------------------------
@app.get("/")
def index():
    return render_template_string(INDEX_HTML)

@app.post("/api/start")
def api_start():
    data = request.get_json(force=True)
    token = (data.get("token") or "").strip()
    startDate = (data.get("startDate") or "").strip()
    endDate = (data.get("endDate") or "").strip()

    if not token or not startDate or not endDate:
        return jsonify({"error":"missing parameters"}), 400

    job_id = uuid.uuid4().hex
    job = Job(token=token, start=startDate, end=endDate)
    with JOBS_LOCK:
        JOBS[job_id] = job

    t = threading.Thread(target=run_job_bg, args=(job_id,), daemon=True)
    t.start()

    return jsonify({"job_id": job_id})

@app.get("/api/progress/<job_id>")
def api_progress(job_id):
    job = JOBS.get(job_id)
    if not job:
        return jsonify({"error":"job not found"}), 404
    with JOBS_LOCK:
        data = dict(
            total=job.total,
            done=job.done,
            finished=job.finished,
            stopped=job.stop_flag,
            error=job.error,
            state=job.state
        )
    return jsonify(data)

@app.get("/api/rows/<job_id>")
def api_rows(job_id):
    job = JOBS.get(job_id)
    if not job:
        return jsonify({"error":"job not found"}), 404
    with JOBS_LOCK:
        rows = list(job.rows)  # copy
    return jsonify({"rows": rows})

@app.post("/api/stop/<job_id>")
def api_stop(job_id):
    job = JOBS.get(job_id)
    if not job:
        return jsonify({"error":"job not found"}), 404
    with JOBS_LOCK:
        job.stop_flag = True
        job.state = "กำลังยกเลิก..."
    return jsonify({"ok": True})

if __name__ == "__main__":
    # dev server สำหรับ Colab
    app.run(host="0.0.0.0", port=5000, debug=False)


Writing app.py


In [ ]:
from pyngrok import ngrok

# ใส่ ngrok authtoken ของคุณตรงนี้
NGROK_TOKEN = "2VLMZ4oBIu3VXGKWhP5SNkaXEbh_27rMTdT2efum4AzpZJDis"

ngrok.set_auth_token(NGROK_TOKEN)


In [ ]:
import threading, time
from pyngrok import ngrok
from app import app

# เปิด tunnel ไปยัง port 5000
public_url = ngrok.connect(5000)
print("ngrok URL:", public_url)

# รัน Flask ใน thread แยก
def run_flask():
    app.run(host="0.0.0.0", port=5000)

thread = threading.Thread(target=run_flask)
thread.start()

# รอให้ Flask start
time.sleep(2)


ngrok URL: NgrokTunnel: "https://04475ffa1a01.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app 'app'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
